In [285]:
#download and install CBC solver
#%pip install pyomo-windows

# from pyomo_windows.solvers import DownloadSolvers
# downloader = DownloadSolvers()
# downloader.download_cbc()

In [286]:
import matplotlib.pyplot as plt
import pandas as pd
import importlib
from pyomo.environ import *
%matplotlib inline

import ParameterDataLoader as ParameterDataLoader_Module
import MultiCriteriaMIPModel as MultiCriteriaMIPModel_Module

importlib.reload(ParameterDataLoader_Module) # in case of updates
importlib.reload(MultiCriteriaMIPModel_Module) # in case of updates

from ParameterDataLoader import ParameterDataLoader
from MultiCriteriaMIPModel import MultiCriteriaMIPModel

H_FIXED_MINUTES = 480
ALPHA = 1.0 #makespan weight
BETA = 1.0 #Operator activation weight 
BIG_M = 10000

### Load Mission Batch Datasets
For now, we introduce some simplifications:
- we don't consider the priority of mission, neither the area of mission.
- we don't consider the operational area of forklift.
- we don't consider the tare of mission's pallet (UDC). 
- since we don't consider the Z axis, so we have to suppose that the loading & unloading speeds are equal for all mission types; therefore, there's no need to consider them.
- the travel time is calculated using forklift's overall average speed, not for each single forklift.

*Mission types are {loading, unloading, internal repositioning}.

In [287]:
MISSION_BATCH_DIR = "./datasets/Batch100M_distanced.csv"
UDC_TYPES_DIR = "./datasets/WM_UDC_TYPE.csv"
MISSION_BATCH_TRAVEL_DIR = "./datasets/Batch100M_travel_distanced.csv"
FORK_LIFTS_DIR = "./datasets/ForkLifts.csv"
#MISSION_TYPES_DIR = "./datasets/MissionTypes.csv"

#with mission TP_UDC, we can retreive relative width and length from mission types
#mission_batch_features = ['CD_MISSION', 'TP_MISSION', 'FROM_X', 'FROM_Y', 'TO_X', 'TO_Y', 'TP_UDC', 'DISTANCE']
mission_batch_features = ['CD_MISSION', 'FROM_X', 'FROM_Y', 'TO_X', 'TO_Y', 'TP_UDC', 'DISTANCE']
udc_types_features = ['TP_UDC', 'WIDTH', 'LENGTH']
mission_batch_travel_features = ['CD_MISSION_1', 'CD_MISSION_2', 'FROM_X', 'FROM_Y', 'TO_X', 'TO_Y', 'DISTANCE']
#fork_lifts_features = ['OID', 'FORK_WIDTH', 'FORK_LEGNTH', 'SPEED', 'SPEED_WITH_LOAD', 'UP_SPEED', 'UP_SPEED_WITH_LOAD', 'DOWN_SPEED', 'DOWN_SPEED_WITH_LOAD']
fork_lifts_features = ['OID', 'FORK_WIDTH', 'FORK_LEGNTH', 'SPEED', 'SPEED_WITH_LOAD']
#mission_types_features = ['TP_MISSION', 'DSC_MISSION']

mission_batch_df = pd.read_csv(MISSION_BATCH_DIR)[mission_batch_features]
udc_types_df = pd.read_csv(UDC_TYPES_DIR)[udc_types_features]
mission_batch_travel_df = pd.read_csv(MISSION_BATCH_TRAVEL_DIR)[mission_batch_travel_features]
fork_lifts_df = pd.read_csv(FORK_LIFTS_DIR)[fork_lifts_features]
#mission_types_df = pd.read_csv(MISSION_TYPES_DIR)[mission_types_features]

mission_batch_df.head()

,CD_MISSION,FROM_X,FROM_Y,TO_X,TO_Y,TP_UDC,DISTANCE
0,3911722,246,426,194,373,1.0,59
1,3911727,246,426,194,373,1.0,59
2,3911733,246,426,194,373,1.0,59
3,3911740,262,329,194,373,1.0,103
4,3911742,246,426,194,373,1.0,59


In [288]:
udc_types_df.head()

,TP_UDC,WIDTH,LENGTH
0,8,1.05,1.05
1,13,1.23,1.43
2,14,1.74,1.83
3,15,0.75,1.70
4,16,0.75,2.50


##### Mission feature units
- distance, width and length in meter.
Note that the distance is already pre-calculated using external program by applying A* on features {FORM_X, FROM_Y, TO_X, TO_Y}. The external progam has considered an image that describe the real-world warehouse map, then the path is estimated by A* after scaling image's pixels to calculate the distance approximately.  

*A future efficient approach could be through saving the warehouse map on a geografic system exploiting GIS queries for distance and path calculations.

In [289]:
BASE_MISSION = [0, 0, 0, 0, 0, 0, 0, 0]  #virtual base mission for operators to start and end their routes

mission_batch_df.TP_UDC.fillna(1, inplace=True)
mission_batch_df = pd.merge(mission_batch_df, udc_types_df, on='TP_UDC')
mission_batch_df.drop(columns=['TP_UDC'], inplace=True)
mission_batch_df_with_base = pd.concat([pd.DataFrame([BASE_MISSION], columns=mission_batch_df.columns), mission_batch_df], ignore_index=True)
mission_batch_df.head()

C:\Users\abudo\AppData\Local\Temp\ipykernel_17568\408980915.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  mission_batch_df.TP_UDC.fillna(1, inplace=True)


,CD_MISSION,FROM_X,FROM_Y,TO_X,TO_Y,DISTANCE,WIDTH,LENGTH
0,3911722,246,426,194,373,59,0.8,1.2
1,3911727,246,426,194,373,59,0.8,1.2
2,3911733,246,426,194,373,59,0.8,1.2
3,3911740,262,329,194,373,103,0.8,1.2
4,3911742,246,426,194,373,59,0.8,1.2


In [290]:
mission_batch_travel_df.head()

,CD_MISSION_1,CD_MISSION_2,FROM_X,FROM_Y,TO_X,TO_Y,DISTANCE
0,3911722,3911727,194,373,246,426,59
1,3911727,3911722,194,373,246,426,59
2,3911722,3911733,194,373,246,426,59
3,3911733,3911722,194,373,246,426,59
4,3911722,3911740,194,373,262,329,99


##### Fork Lift feature units
- width & length in meter.
- speed in meter/minute.

In [291]:
fork_lifts_df.head()

,OID,FORK_WIDTH,FORK_LEGNTH,SPEED,SPEED_WITH_LOAD
0,1,1.1,1.7,300.0,300.0
1,2,1.1,1.7,300.0,300.0
2,3,1.1,1.7,300.0,300.0
3,4,1.1,1.7,300.0,300.0
4,5,1.1,1.7,300.0,300.0


In [ ]:
parameter_data_loader = ParameterDataLoader(
    mission_batch_df,
    mission_batch_travel_df,
    fork_lifts_df
)

processing_times = parameter_data_loader.get_mission_processing_times()
travel_times = parameter_data_loader.get_mission_travel_times()
skill_scores = parameter_data_loader.get_operator_skill_scores()

In [ ]:
mcmModel= MultiCriteriaMIPModel(mission_batch_df.CD_MISSION.to_list(),
                                 fork_lifts_df.OID.to_list(),
                                 mission_batch_df_with_base.CD_MISSION.to_list(),
                                 processing_times,
                                 travel_times,
                                 skill_scores,
                                 H_FIXED_MINUTES,
                                 ALPHA,
                                 BETA,
                                 BIG_M
                                )

#instance, results = mcmModel.solve("Mip_parameters.dat", "cbc")
#instance, results = mcmModel.solve("Mip_parameters.dat")

isdiscrete() or isfinite()  (deprecated in 5.7) (called from
C:\Users\abudo\AppData\Roaming\Python\Python313\site-
packages\debugpy\_vendored\pydevd\_pydevd_bundle\pydevd_resolver.py:193)
5.7) (called from C:\Users\abudo\AppData\Roaming\Python\Python313\site-
packages\debugpy\_vendored\pydevd\_pydevd_bundle\pydevd_resolver.py:193)
isordered()  (deprecated in 5.7) (called from
C:\Users\abudo\AppData\Roaming\Python\Python313\site-
packages\debugpy\_vendored\pydevd\_pydevd_bundle\pydevd_resolver.py:193)
retrieve the values in a finite set.  (deprecated in 5.7) (called from
C:\Users\abudo\AppData\Roaming\Python\Python313\site-
packages\debugpy\_vendored\pydevd\_pydevd_bundle\pydevd_resolver.py:193)
.ordered_data() to retrieve the values from a finite set in a deterministic
order.  (deprecated in 5.7) (called from
C:\Users\abudo\AppData\Roaming\Python\Python313\site-
packages\debugpy\_vendored\pydevd\_pydevd_bundle\pydevd_resolver.py:193)
(deprecated in 5.7) (called from
C:\Users\abudo\AppD

ValueError: Error retrieving immutable Param value (Alpha):
	The Param value is undefined and no default value is specified.

In [133]:
mcmModel.display_solution(instance)

Model unknown

  Variables:
    y : Size=2, Index=I_max
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          A :     0 :   1.0 :     1 : False : False : Binary
          B :     0 :   1.0 :     1 : False : False : Binary
    z : Size=50, Index=I_max*J_prime*J_prime
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
        ('A', 0, 0) :     0 :  None :     1 : False :  True : Binary
        ('A', 0, 1) :     0 :   0.0 :     1 : False : False : Binary
        ('A', 0, 2) :     0 :   0.0 :     1 : False : False : Binary
        ('A', 0, 3) :     0 :   0.0 :     1 : False : False : Binary
        ('A', 0, 4) :     0 :   1.0 :     1 : False : False : Binary
        ('A', 1, 0) :     0 :   1.0 :     1 : False : False : Binary
        ('A', 1, 1) :     0 :  None :     1 : False :  True : Binary
        ('A', 1, 2) :     0 :   0.0 :     1 : False : False : Binary
        ('A', 1, 3) :     0 :   0.0 :     1 : False : False : Binary
        ('A', 1, 4) :     

In [134]:
solution_data = []
for i in instance.I_max:
    if value(instance.y[i]) == 1: #in case of unfeasible solution, it launches a pyomo error
        #operator i is active
        for j in instance.J_prime:
            for k in instance.J_prime:
                
                #apply the j != k filter to avoid pyomo warning about self-loops
                if j != k:
                    #check if the index (i, j, k) is valid and exists in z
                    if (i, j, k) in instance.z:
                        
                        #check for active flow
                        if value(instance.z[i, j, k]) > 0.5:
                            
                            #add data only for flow between service orders (not flow to/from Base)
                            if j in instance.J and k in instance.J:
                                solution_data.append({
                                    'Operator': i,
                                    'Task': j,
                                    'Start': value(instance.S[j]),
                                    'Finish': value(instance.C[j]),
                                    'Successor': k
                                })

df_schedule = pd.DataFrame(solution_data)
df_schedule = df_schedule.sort_values(by=['Operator', 'Start']).reset_index(drop=True)
print(df_schedule)

  Operator  Task  Start  Finish  Successor
0        A     4    0.0    18.0          1
1        B     2    0.0    10.0          3


In [135]:
routes = {} 
    
# Check for solution status (optional but recommended)
# if not (results.solver.status == SolverStatus.ok and results.solver.termination_condition == TerminationCondition.optimal):
#     print("Solver did not find an optimal solution.")
#     return

# Call the function with your solved instance
# print_optimal_routes(instance)

# 1. Iterate over all potential operators
for i in instance.I_max:
    if value(instance.y[i]) < 0.5: #in case of unfeasible solution, it launches a pyomo error
        continue #skip unactivated operators

    print(f"\n--- Operator {i} (Activated) ---")

    current_node = 0  # Start at the Base node
    route_sequence = []
    is_route_complete = False

    #security counter to prevent infinite loops (should not happen if flow constraints are correct)
    max_steps = len(instance.J) + 2 
    steps = 0

    #loop until the route returns to the Base (k=0)
    while not is_route_complete and steps < max_steps:
        steps += 1
        
        #search for the next step (k) starting from the current node (current_node)
        found_next_step = False
        for k in instance.J_prime:
            if current_node == k:
                continue # Skip self-loop
            
            try:
                #check if the arc (current_node -> k) is active
                if value(instance.z[i, current_node, k]) > 0.5:
                    
                    #calculate travel time for printing
                    travel_time = value(instance.T[current_node, k])
                    
                    #handle Movement Types
                    if k == 0:
                        #final movement: Return to Base
                        route_sequence.append(f"-> Base (Arc: {current_node} -> 0 | Travel: {travel_time:.2f})")
                        is_route_complete = True
                        break #exit the k loop
                    else:
                        #service movement: j -> k
                        start_time = value(instance.S[k])
                        finish_time = value(instance.C[k])
                        proc_time = finish_time - start_time
                        
                        movement_detail = (
                            f"-> Order {k} | Travel: {travel_time:.2f} | Start: {start_time:.2f} | "
                            f"Proc: {proc_time:.2f} | Finish: {finish_time:.2f}"
                        )
                        route_sequence.append(movement_detail)
                        
                        #move to the next node in the sequence
                        current_node = k
                        found_next_step = True
                        break #exit the k loop
                        
            except KeyError:
                #this node pair might not exist in the defined set of z variables (e.g., if filtered by a complex index)
                continue

        if not found_next_step and not is_route_complete:
            print(f"Error: Route stopped unexpectedly at node {current_node} for operator {i}.")
            break
        
    #print the final sequenced route for the operator
    route_string = "\n".join(route_sequence)
    print("Path: Base " + route_string)
    print(f"Total Time (C_last): {value(instance.C_last[i]):.2f}")
    print("-" * 40)


--- Operator A (Activated) ---
Path: Base -> Order 4 | Travel: 7.00 | Start: 0.00 | Proc: 18.00 | Finish: 18.00
-> Order 1 | Travel: 2.00 | Start: 20.00 | Proc: 20.00 | Finish: 40.00
-> Base (Arc: 1 -> 0 | Travel: 6.00)
Total Time (C_last): 46.00
----------------------------------------

--- Operator B (Activated) ---
Path: Base -> Order 2 | Travel: 8.00 | Start: 0.00 | Proc: 10.00 | Finish: 10.00
-> Order 3 | Travel: 5.00 | Start: 15.00 | Proc: 20.00 | Finish: 35.00
-> Base (Arc: 3 -> 0 | Travel: 5.00)
Total Time (C_last): 46.00
----------------------------------------
